In [1]:
import pandas as pd
import json
import os
import openai
from dotenv import load_dotenv
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

In [2]:
load_dotenv()

key = os.getenv("OPENAI_API_KEY")

openai.api_key = key


In [3]:
def role_based_knowledge_generation():

    prompt = (
        "You are an expert in public health sentiment analysis. Explain what sentiment means "
        "and list common linguistic cues in vaccine and mask hesitancy discourse (e.g., sarcasm, "
        "negative adjectives, strong expressions like 'hate' or 'disgust'). Provide examples."
    )
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    
    return response.choices[0].message.content

def sentiment_detection(sentence):

    prompt = f"Does the following sentence express a sentiment regarding vaccines or masks? Answer Yes or No.\nSentence: \"{sentence}\""
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    return response.choices[0].message.content

def sentiment_classification(sentence):

    prompt = f"Classify the sentiment of the following sentence as Positive, Negative, or Neutral:\n\"{sentence}\""
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    return response.choices[0].message.content

def key_phrase_extraction(sentence):

    prompt = f"Identify the key phrases or words that indicate the sentiment in the sentence \"{sentence}\" and briefly explain why."
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    return response.choices[0].message.content

def sentiment_summary(sentence):

    prompt = f"Generate a concise summary of the sentiment expressed in the sentence \"{sentence}\""
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    return response.choices[0].message.content

def rbic_sentiment_pipeline(sentence):

    knowledge = role_based_knowledge_generation()
    detection = sentiment_detection(sentence)
    classification = sentiment_classification(sentence)
    key_phrases = key_phrase_extraction(sentence)
    summary = sentiment_summary(sentence)
    

    label_map = {
    "Positive": 1,
    "Neutral": 0,
    "Negative": -1
        }

    # Convert classification string to numeric label
    numeric_label = label_map.get(classification.strip(), None)    

    output = {
        "knowledge_base": knowledge.strip(),
        "sentiment_detected": detection.strip(),
        "sentiment_label": numeric_label,
        "key_phrases": key_phrases.strip(),
        "summary": summary.strip()
    }
    return output

In [ ]:
if __name__ == "__main__":
    
    df = pd.read_csv("posts_to_label.csv")  

    df = df[df['selftext'].notnull()]

    def get_rbic_prediction(text):
        try:
            result = rbic_sentiment_pipeline(text)
            return result['sentiment_label']  # Only get numeric label
        except Exception as e:
            print(f"Error processing text: {text[:30]}... -> {e}")
            return None

    print("Running RBIC sentiment pipeline...")
    df['predicted_label'] = df['selftext'].apply(get_rbic_prediction)
    
    df = df[df['predicted_label'].notnull()]
    df['manual_label'] = df['manual_label'].astype(int)

    
    print("\nClassification Report:")
    print(classification_report(df['manual_label'], df['predicted_label'], labels=[-1, 0, 1], target_names=['Negative', 'Neutral', 'Positive']))

    f1 = f1_score(df['manual_label'], df['predicted_label'], average='weighted')
    print(f"\nWeighted F1 Score: {f1:.4f}")

Running RBIC sentiment pipeline...

Classification Report:
              precision    recall  f1-score   support

    Negative       1.00      1.00      1.00         1
     Neutral       0.00      0.00      0.00         0
    Positive       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.33      0.33      0.33         2
weighted avg       0.50      0.50      0.50         2


Weighted F1 Score: 0.5000


c:\DDrive\Work\1 UNSW\2 Course Work\25T1\Thesis A\Code\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\DDrive\Work\1 UNSW\2 Course Work\25T1\Thesis A\Code\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\DDrive\Work\1 UNSW\2 Course Work\25T1\Thesis A\Code\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(aver

In [69]:
if __name__ == "__main__":
    
    df = pd.read_csv("posts_to_label.csv")  

    df = df[df['selftext'].notnull()]

    def get_rbic_prediction(text):
        try:
            result = rbic_sentiment_pipeline(text)
            return result['sentiment_label']  # Only get numeric label
        except Exception as e:
            print(f"Error processing text: {text[:30]}... -> {e}")
            return None

    print("Running RBIC sentiment pipeline...")
    df['predicted_label'] = df['selftext'].apply(get_rbic_prediction)
    
    df = df[df['predicted_label'].notnull()]
    df['manual_label'] = df['manual_label'].astype(int)

    
    print("\nClassification Report:")
    print(classification_report(df['manual_label'], df['predicted_label'], labels=[-1, 0, 1], target_names=['Negative', 'Neutral', 'Positive']))

    f1 = f1_score(df['manual_label'], df['predicted_label'], average='weighted')
    print(f"\nWeighted F1 Score: {f1:.4f}")
    

Running RBIC sentiment pipeline...


KeyboardInterrupt: 